In [1]:
#import CUBRIDdb
import pymysql
import pandas as pd
from datetime import datetime, timedelta
import sys

In [2]:
#sys.stdout = open('D:\python\error_log.txt','a')

In [3]:
#today = datetime.now()
today = datetime(2020, 11, 30)
today = today.strftime('%Y-%m-%d')#ex)2019-03-14

In [4]:
conn = pymysql.connect(
    user='egov39exbd', 
    passwd='579nof83', 
    host='172.17.17.165', 
    db='egov39exbd', 
    charset='utf8'
)
cur = conn.cursor(pymysql.cursors.DictCursor)
select_query= """\
    SELECT
        cstmr_no,
        mrnr_no,
        SUM(smrt_mtinsp_hour_usgqty) AS usgqty,
        MAX(smrt_mtinsp_indctpnt) AS max_indctpnt, 
        MIN(smrt_mtinsp_indctpnt) AS min_indctpnt
    FROM
        asmartmiteringinfo
    WHERE mrnr_no = 'A00'
    AND creat_stle = 'N'
    AND mtinsp_dt >= DATE_ADD('{today}', interval -3 MONTH)
    GROUP BY cstmr_no, mrnr_no
""".format(today=today)
select_query = select_query.replace("\n\t"," ")
select_query = select_query.replace("\n"," ")
cur.execute(select_query)
# HAVING usgqty = 0

rows = cur.fetchall()
df = pd.DataFrame(rows)
df.columns=['cstmr_no', 'mrnr_no', 'usgqty','max_indctpnt','min_indctpnt']

In [5]:
df

,cstmr_no,mrnr_no,usgqty,max_indctpnt,min_indctpnt
0,1952000001,A00,24.705,104.428,78.387
1,1965000030,A00,19.057,55.624,34.553
2,1965000051,A00,20.437,97.235,72.184
3,1967000092,A00,5.937,22.738,16.738
4,1969000139,A00,14.778,76.906,58.735
5,1969000159,A00,21.187,111.799,85.419
6,1970000004,A00,9.551,19.575,10.025
7,1970000026,A00,8.235,39.260,29.054
8,1970000044,A00,4.402,299.843,294.421
9,1970000075,A00,76.453,431.410,329.814


In [6]:
df['cal'] = df['max_indctpnt']-df['min_indctpnt']

In [7]:
type(df['max_indctpnt'])

pandas.core.series.Series

In [8]:
df['cal']

0     26.041
1     21.071
2     25.051
3      6.000
4     18.171
5     26.380
6      9.550
7     10.206
8      5.422
9    101.596
Name: cal, dtype: object

In [9]:
df_cal = df[df['cal'] > 6] #3개월 사용량 합이 0인 집 대상 (0인집이 없어 일단 6으로 걸어봄)

In [10]:
df_cal

,cstmr_no,mrnr_no,usgqty,max_indctpnt,min_indctpnt,cal
0,1952000001,A00,24.705,104.428,78.387,26.041
1,1965000030,A00,19.057,55.624,34.553,21.071
2,1965000051,A00,20.437,97.235,72.184,25.051
4,1969000139,A00,14.778,76.906,58.735,18.171
5,1969000159,A00,21.187,111.799,85.419,26.380
6,1970000004,A00,9.551,19.575,10.025,9.550
7,1970000026,A00,8.235,39.260,29.054,10.206
9,1970000075,A00,76.453,431.410,329.814,101.596
